# 🔨 Tech Deal Forge - ML Training Notebook

**Google Colab ML Training Script for Deal Quality Prediction**

---

## 📋 Workflow

1. ✅ **Upload to Google Colab** - Open this notebook in Colab
2. ✅ **Upload CSV file** - Export from `deals.db` using `export_deals_for_ml.py`
3. ✅ **Run all cells** - Execute the entire notebook
4. ✅ **Download trained model** - Get the `.joblib` file
5. ✅ **Deploy to Streamlit** - Place model in project folder

---

## 🎯 What This Notebook Does

- Loads deals data from CSV (exported from SQLite database)
- Engineers features (website encoding, categories, temporal)
- Creates target variable (deal quality score 0-100)
- Trains Random Forest model
- Evaluates performance (R², RMSE, MAE)
- Visualizes feature importance
- Exports trained model for Streamlit dashboard

---

**Author:** Tech Deal Forge  
**Date:** November 2024  
**Model Type:** Random Forest Regressor  
**Purpose:** Predict deal quality based on discount, rating, reviews, and other features

## 📤 Step 1: Upload CSV Data

Upload your deals CSV file exported from the database.

**To generate this CSV locally:**
```bash
python export_deals_for_ml.py
```

This creates: `output/ml_training_data_YYYYMMDD_HHMMSS.csv`

**Upload that file here ⬇️**

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("🔨 TECH DEAL FORGE - ML MODEL TRAINING")
print("=" * 60)
print("\n📤 Upload your deals CSV file:")
print("   (Look for 'ml_training_data_*.csv' in your output folder)")

uploaded = files.upload()

if not uploaded:
    raise ValueError("❌ No file uploaded! Please upload a CSV file.")

filename = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {filename}")

# Load data
df = pd.read_csv(filename)
print(f"✅ Loaded {len(df):,} deals")

## 📊 Step 2: Data Quality Report

Check the data before training to ensure it's complete and valid.

In [ ]:
print("\n" + "=" * 60)
print("📊 DATA QUALITY REPORT")
print("=" * 60)

print(f"\n📈 Dataset Overview:")
print(f"   - Total rows: {len(df):,}")
print(f"   - Columns: {len(df.columns)}")
print(f"   - Date range: {df['scraped_at'].min()} → {df['scraped_at'].max()}")
print(f"   - Websites: {', '.join(df['website'].value_counts().index.tolist())}")
print(f"   - Categories: {df['category'].nunique()} unique")

print(f"\n💰 Price Statistics:")
df['price_numeric'] = pd.to_numeric(df['price_numeric'], errors='coerce')
valid_prices = df['price_numeric'].dropna()
if len(valid_prices) > 0:
    print(f"   - Min price: ${valid_prices.min():.2f}")
    print(f"   - Max price: ${valid_prices.max():.2f}")
    print(f"   - Avg price: ${valid_prices.mean():.2f}")
    print(f"   - Median price: ${valid_prices.median():.2f}")

print(f"\n🔢 Data Completeness:")
critical_cols = ['price_numeric', 'discount_percent', 'rating', 'reviews_count', 'category', 'website']
for col in critical_cols:
    if col in df.columns:
        missing = df[col].isna().sum()
        pct = (missing / len(df)) * 100
        status = "✅" if pct < 20 else "⚠️" if pct < 50 else "❌"
        print(f"   {status} {col}: {pct:.1f}% missing ({missing:,} rows)")

# Display first few rows
print(f"\n📋 Sample Data (first 5 rows):")
display(df.head())

## 🔧 Step 3: Feature Engineering

Prepare features that match your `ml_integration.py` structure:
- Clean numeric columns
- Encode websites (one-hot)
- Encode categories (multi-label)
- Create temporal features
- Generate target variable (deal quality score)

In [ ]:
def prepare_training_data(df):
    """
    Prepare features matching ml_integration.py
    This ensures compatibility with your Streamlit dashboard
    """
    
    # Clean numeric columns
    df['price_numeric'] = pd.to_numeric(df['price_numeric'], errors='coerce')
    df['discount_percent'] = pd.to_numeric(df['discount_percent'], errors='coerce').fillna(0)
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0)
    df['reviews_count'] = pd.to_numeric(df['reviews_count'], errors='coerce').fillna(0)
    
    # Website encoding (one-hot)
    df['website_bestbuy'] = (df['website'] == 'bestbuy').astype(int)
    df['website_slickdeals'] = (df['website'] == 'slickdeals').astype(int)
    df['website_newegg'] = (df['website'] == 'newegg').astype(int)
    
    # Category encoding (multi-label possible)
    df['category_gaming'] = df['category'].str.contains('gaming|game', case=False, na=False).astype(int)
    df['category_laptop'] = df['category'].str.contains('laptop|notebook', case=False, na=False).astype(int)
    df['category_monitor'] = df['category'].str.contains('monitor|display', case=False, na=False).astype(int)
    df['category_electronics'] = df['category'].str.contains('electronics|tech', case=False, na=False).astype(int)
    
    # Temporal features
    df['scraped_at'] = pd.to_datetime(df['scraped_at'])
    df['day_of_week'] = df['scraped_at'].dt.dayofweek
    df['month'] = df['scraped_at'].dt.month
    df['is_weekend'] = (df['scraped_at'].dt.dayofweek >= 5).astype(int)
    
    # Historical features (simplified - no lookback for training)
    df['price_vs_avg'] = 1.0
    df['price_vs_min'] = 1.0
    df['times_seen'] = 1
    df['price_std'] = 0.0
    df['recent_trend'] = 0.0
    
    # CREATE TARGET: Deal Quality Score (0-100)
    max_discount = df['discount_percent'].max() if df['discount_percent'].max() > 0 else 1
    
    df['deal_quality_score'] = (
        (df['discount_percent'] / max_discount * 40) +  # 40% weight on discount
        (df['rating'] / 5.0 * 30) +                     # 30% weight on rating
        (np.clip(df['reviews_count'], 0, 100) / 100 * 30)  # 30% weight on reviews
    )
    
    # Fill missing scores with median
    median_score = df['deal_quality_score'].median()
    df['deal_quality_score'] = df['deal_quality_score'].fillna(median_score)
    
    # CRITICAL: Feature list MUST match ml_integration.py prepare_features()
    feature_cols = [
        'price_numeric', 'discount_percent', 'rating', 'reviews_count',
        'website_bestbuy', 'website_slickdeals',
        'category_gaming', 'category_laptop', 'category_monitor',
        'day_of_week', 'month', 'is_weekend',
        'price_vs_avg', 'price_vs_min', 'times_seen', 'price_std', 'recent_trend'
    ]
    
    X = df[feature_cols].fillna(0)
    y = df['deal_quality_score']
    
    # Remove any rows with NaN in target
    valid_idx = ~y.isna()
    X = X[valid_idx]
    y = y[valid_idx]
    
    return X, y, feature_cols

# Prepare features
print("\n" + "=" * 60)
print("🔧 FEATURE ENGINEERING")
print("=" * 60)

X, y, feature_names = prepare_training_data(df)

print(f"\n✅ Training data prepared:")
print(f"   - Samples: {X.shape[0]:,} deals")
print(f"   - Features: {X.shape[1]}")
print(f"   - Target range: {y.min():.1f} - {y.max():.1f}")
print(f"\n📋 Features used:")
for i, feat in enumerate(feature_names, 1):
    print(f"   {i:2d}. {feat}")

## 🤖 Step 4: Train Model

Train a Random Forest Regressor to predict deal quality scores.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Split data (80/20 train/test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\n" + "=" * 60)
print("🤖 MODEL TRAINING")
print("=" * 60)

print(f"\n📊 Dataset split:")
print(f"   - Training: {len(X_train):,} deals ({len(X_train)/len(X)*100:.1f}%)")
print(f"   - Testing: {len(X_test):,} deals ({len(X_test)/len(X)*100:.1f}%)")

# Train Random Forest
print(f"\n🌲 Training Random Forest Regressor...")
model = RandomForestRegressor(
    n_estimators=100,      # Number of trees
    max_depth=10,          # Prevent overfitting
    min_samples_split=5,   # Require at least 5 samples to split
    random_state=42,       # Reproducibility
    n_jobs=-1              # Use all CPU cores
)

model.fit(X_train, y_train)
print(f"✅ Model trained successfully!")

## 📈 Step 5: Evaluate Performance

Check how well the model performs on unseen test data.

In [ ]:
print("\n" + "=" * 60)
print("📈 MODEL PERFORMANCE")
print("=" * 60)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Metrics
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"\n🎯 Training Set Performance:")
print(f"   - R² Score: {r2_train:.3f}")

print(f"\n🎯 Test Set Performance:")
print(f"   - R² Score: {r2_test:.3f} (higher is better, max 1.0)")
print(f"   - RMSE: {rmse_test:.2f} points")
print(f"   - MAE: {mae_test:.2f} points (avg error)")

# Interpretation
if r2_test > 0.7:
    print(f"\n✅ EXCELLENT - Model explains {r2_test*100:.1f}% of variance")
elif r2_test > 0.5:
    print(f"\n✅ GOOD - Model explains {r2_test*100:.1f}% of variance")
elif r2_test > 0.3:
    print(f"\n⚠️  FAIR - Model explains {r2_test*100:.1f}% of variance")
else:
    print(f"\n⚠️  LIMITED - Model explains only {r2_test*100:.1f}% of variance")
    print(f"   Consider collecting more diverse data or adding features")

# Check for overfitting
if r2_train - r2_test > 0.2:
    print(f"\n⚠️  WARNING: Possible overfitting detected")
    print(f"   Training R²: {r2_train:.3f} | Test R²: {r2_test:.3f}")
    print(f"   Consider: more data, simpler model, or regularization")

## 🔍 Step 6: Feature Importance

See which features matter most for predicting deal quality.

In [ ]:
import matplotlib.pyplot as plt

print("\n" + "=" * 60)
print("🔍 FEATURE IMPORTANCE")
print("=" * 60)

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n📊 Top Features (what matters most):")
print(importance_df.head(10).to_string(index=False))

# Plot feature importance
plt.figure(figsize=(12, 6))
plt.barh(importance_df['feature'][:10], importance_df['importance'][:10])
plt.xlabel('Importance Score', fontsize=12)
plt.title('Top 10 Most Important Features for Deal Quality Prediction', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 🎯 Step 7: Sample Predictions

Test the model on real examples to see how accurate it is.

In [ ]:
print("\n" + "=" * 60)
print("🎯 SAMPLE PREDICTIONS")
print("=" * 60)

# Test on 10 random examples
n_samples = min(10, len(X_test))
sample_idx = np.random.choice(len(X_test), n_samples, replace=False)
test_samples = X_test.iloc[sample_idx]
test_actuals = y_test.iloc[sample_idx]
test_predictions = model.predict(test_samples)

comparison = pd.DataFrame({
    'Actual Score': test_actuals.values,
    'Predicted Score': test_predictions,
    'Difference': test_predictions - test_actuals.values,
    'Error %': ((test_predictions - test_actuals.values) / test_actuals.values * 100)
})

print(f"\n{comparison.to_string(index=False)}")
print(f"\nAverage absolute error: {np.abs(comparison['Difference']).mean():.2f} points")

# Visual comparison
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, alpha=0.5, s=20)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Deal Quality Score', fontsize=12)
plt.ylabel('Predicted Deal Quality Score', fontsize=12)
plt.title('Predicted vs Actual Deal Quality Scores', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 💾 Step 8: Save and Download Model

Export the trained model to use in your Streamlit dashboard.

In [ ]:
import joblib
from datetime import datetime

print("\n" + "=" * 60)
print("💾 SAVING MODEL")
print("=" * 60)

# Generate filename with timestamp
model_filename = f"deal_predictor_{datetime.now().strftime('%Y%m%d_%H%M%S')}.joblib"

# Save model with metadata
model_data = {
    'model': model,
    'feature_names': feature_names,
    'r2_score': r2_test,
    'rmse': rmse_test,
    'mae': mae_test,
    'trained_on': datetime.now().isoformat(),
    'training_samples': len(X_train),
    'test_samples': len(X_test)
}

joblib.dump(model_data, model_filename)

print(f"\n✅ Model saved: {model_filename}")
print(f"\n📦 Model Package Contents:")
print(f"   - Model: Random Forest Regressor")
print(f"   - Features: {len(feature_names)}")
print(f"   - R² Score: {r2_test:.3f}")
print(f"   - RMSE: {rmse_test:.2f}")
print(f"   - Training samples: {len(X_train):,}")

# Download the model
print(f"\n⬇️  Downloading model...")
files.download(model_filename)

## ✅ Training Complete! Next Steps

Follow these instructions to deploy your model.

In [ ]:
print("\n" + "=" * 60)
print("✅ TRAINING COMPLETE!")
print("=" * 60)

print(f"\n🚀 Next Steps:")
print(f"\n1️⃣  Place downloaded model in your project root folder:")
print(f"    📁 Senior-Project-Tech-Deal-Forge/")
print(f"       └── {model_filename}")

print(f"\n2️⃣  Test locally in Streamlit:")
print(f"    streamlit run streamlit_dashboard.py")
print(f"    → Go to '🤖 AI Predictions' tab")
print(f"    → Enter model path: {model_filename}")

print(f"\n3️⃣  Deploy to Streamlit Cloud:")
print(f"    git add {model_filename}")
print(f"    git commit -m 'Add trained ML model'")
print(f"    git push origin main")

print(f"\n📊 Model Performance Summary:")
print(f"   ✅ R² Score: {r2_test:.3f}")
print(f"   ✅ Average error: {mae_test:.2f} points")
print(f"   ✅ Trained on: {len(X_train):,} deals")

print(f"\n💡 Tips:")
print(f"   - Higher R² = better predictions (max 1.0)")
print(f"   - Retrain monthly with new data for better accuracy")
print(f"   - Feature importance shows what drives deal quality")

print(f"\n🎉 Your ML model is ready to predict deal quality!")